In [2]:
import numpy as np
import pandas as pd
from torch import nn, optim #adam,gcd
import torch.nn.functional as F
import torch.utils.data as data
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('clinical_with_header.csv')
x = data.iloc[:,1:].values
y = data.iloc[:,0].values
print(x.shape)
print(y.shape)
print(x)


(974, 111)
(974,)
[['TCGA-3C-AALI' '2014-7-28' nan ... '[Not Available]' '3C' 'Breast']
 ['TCGA-3C-AALJ' '2014-7-28' 'NO' ... '[Not Available]' '3C' 'Breast']
 ['TCGA-3C-AALK' '2014-7-28' 'NO' ... '[Not Available]' '3C' 'Breast']
 ...
 ['TCGA-XX-A89A' '2014-2-21' 'NO' ... '[Not Available]' 'XX' 'Breast']
 ['TCGA-Z7-A8R5' '2014-7-9' 'NO' ... '[Not Available]' 'Z7' 'Breast']
 ['TCGA-Z7-A8R6' '2014-7-9' 'NO' ... '[Not Available]' 'Z7' 'Breast']]


In [4]:
gender = data['gender']
d = pd.DataFrame(gender)
d

,gender
0,FEMALE
1,FEMALE
2,FEMALE
3,FEMALE
4,FEMALE
...,...
969,FEMALE
970,FEMALE
971,FEMALE
972,FEMALE


In [5]:
def load_clinical(patients):
        threshold = 365
        binary = {}
        death = {}
        times = {}
        clinical_df = patients
        itr = 0
        for pid in patients.index:
            itr += 1
            # assert pid not in clinical_df.index, f"Invalid Patient ID <{pid}>"
            curr_status = clinical_df.loc[pid]['vital_status']
            num_days = 0
            if curr_status == 'Alive':
                num_days = clinical_df.loc[pid]['last_contact_days_to']
                if num_days in ['[Discrepancy]', '[Not Available]'] :
                    continue
                death[pid] = 0
                times[pid] = num_days
                binary[pid] = 1*(int(num_days) > threshold)
            elif curr_status == 'Dead':
                num_days = clinical_df.loc[pid]['death_days_to']
                if num_days == '[Not Available]':
                    continue
                death[pid] = 1
                times[pid] = num_days
                binary[pid] = 1*(int(num_days) > threshold)
            else:
                print(pid)
                
        labels = []
        for idx in death.keys():
            labels.append(tuple((bool(int(death[idx])), int(times[idx]))))
        dt1=np.dtype(('bool,float'))
        labels = np.array(labels, dtype=dt1)

        # print(list(binary.values()))
        return np.array(list(binary.values())), np.array(death), np.array(times), np.array(labels)

In [6]:
y = load_clinical(data)
print(y[0].shape)


(974,)


In [9]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


torch.Size([640, 1])
torch.Size([160, 1])
torch.Size([640, 1])
torch.Size([160, 1])


In [7]:
class FullyConnected(nn.Module):
        def __init__(self, in_dim=1, hiddens = [8], out_dim=1) -> None:
            super(FullyConnected, self).__init__()
            self.layers = None
            if len(hiddens)>0:
                self.layers = nn.Sequential(
                    nn.Linear(in_features=in_dim, out_features=hiddens[0]),
                    nn.ReLU()
                )
                
                for i in range(1, len(hiddens)):
                    self.layers.add_module(f"fc{i}", nn.Linear(in_features=hiddens[i-1], out_features=hiddens[i]))
                    self.layers.add_module(f"relu{i}", nn.ReLU())
            self.out = nn.Linear(in_features=hiddens[-1], out_features=out_dim)
            self.out.weight.data.zero_()
            self.out.bias.data.zero_()
        def forward(self, x):
            if self.layers is not None:
                x = self.layers(x)
            x = self.out(x)
            return x


                        



In [12]:
from turtle import forward
import matplotlib.pylab as plt
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torchinfo import summary

class FullyConnected(nn.Module):
    def __init__(self, in_dim=1, hiddens = [8], out_dim=1) -> None:
        super(FullyConnected, self).__init__()
        self.layers = None
        if len(hiddens)>0:
            self.layers = nn.Sequential(
                nn.Linear(in_features=in_dim, out_features=hiddens[0]),
                nn.ReLU()
            )
            for i in range(1,len(hiddens)):
                self.layers.add_module(f'H{i}',nn.Linear(hiddens[i-1], hiddens[i]))   
                self.add_module('relu',nn.ReLU())
            self.layers.add_module('Out',nn.Linear(hiddens[-1], out_dim))
        else:
            self.layers = nn.Sequential(
                nn.Linear(in_features=in_dim, out_features=out_dim),
                #nn.ReLU()
            )
        # self.l1 = nn.Linear(in_dim, hiddens[0])
        # self.l2 = nn.Linear(hiddens[0], hiddens[1])
        # #....
        # self.ln = nn.Linear(hiddens[-1], out_dim)
        # self.relu = nn.ReLU()
        
    def forward(self, x):
        # z_l1 = self.l1(x)
        # z_l1 = self.relu(z_l1)
        # z_l2 = self.l2(z_l1)
        # z_l2 = self.relu(z_l2)
        # #....
        # y_hat = self.ln(z_ln_1)
        # y_hat = self.relu(y_hat)
        y_hat = self.layers(x)
        return y_hat

if __name__ == '__main__':
    import os 
    # os.environ["CUDA_VISIBLE_DEVICES"]="-1"
    # torch.cuda.is_available = lambda : True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    import time
    points = np.linspace(-np.pi, np.pi, 1000)
    # y = np.sin(points)
    y_noise = np.sin(points)+0.3*np.random.randn(1000)

    data = np.c_[points.ravel(),y_noise.ravel()]
    # X_train, X_test, y_train, y_test = train_test_split(points, y_noise, test_size=0.2, random_state=1)

    # X_train = X_train.reshape(-1,1)
    # y_train = y_train.reshape(-1,1)
    # X_test = X_test.reshape(-1,1)
    # y_test = y_test.reshape(-1,1)

    epochs = 10000
    lr = 0.001
    batch_size = 100
    myNetword = FullyConnected(1,[20],1).to(device)
    summary(myNetword, (batch_size,1,1)) 
    loss = nn.MSELoss().to(device)
    optim = SGD(myNetword.parameters(),lr=lr)
    # x = torch.from_numpy(X_train).float().to(device)
    # y = torch.from_numpy(y_train).float().to(device)
    x = torch.from_numpy(data[:,0]).float().to(device)
    y = torch.from_numpy(data[:,1]).float().to(device)
    losses = []
    best_loss = 10e20
    print(device)
    start = time.time()
    for epoch in range(1, epochs+1):
        # print(f'\r Epoch [{epoch}/{epochs}] ', end='')
        # y_hat = myNetword(x)

        error = loss(y_hat, y)
        losses.append(error.item())
        optim.zero_grad()
        error.backward()
        optim.step()
        #if error<best_loss:
        #    torch.save(myNetword.state_dict(),'best_model.pt')
        #    best_loss = error
        #    # print('\t Model saved.')
        if epoch % 100 == 0:
           print(f'\rEpoch [{epoch}/{epochs}]: current loss->{error.item():0.5f}, '
                 f'avg loss->{np.mean(losses):.5f}', end='')
    print(f'\nTotal time=> {time.time()-start} ')
    plt.plot(list(range(len(losses))), losses)
    plt.show()
    x_plot = np.linspace(points.min(), points.max(), 1000)
    y_plot = myNetword(torch.from_numpy(x_plot.reshape(-1,1)).float().to(device))
    plt.figure()
    plt.scatter(x.numpy(),y.numpy(),s=2.5)
    plt.plot(x_plot,y_plot.detach().numpy())
    plt.show()

cpu


RuntimeError: size mismatch, got 20, 20x1,1000